In [19]:
import pandas as pd
import numpy as np
from fractions import Fraction

# This algorith is the impelmentation of the strategy developed for the Personal Reocmendation 
Analysys. 

The concepts of lift, conffidence and supoort  are defined for a gorups of costumers that have products in common with the objective client who the recomendations are designed. We briefly define the process of selcting the group and the computations of the parameters give.

1. Given a client $p$ derine the enviroment $G(p)$ as the set of clients that has at least one product in comon with client $p$.
2. Let $A$ a given product. Define the cardinality og $A$, $card(A)=$ number of client that has the article $A$.
3. Define the conffidence $P(A,p)=card(A)/|G(p)|$.

In [3]:
import warnings 
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 

#######  |§4||♫♀♫☼♀♫|☼↑♫☼♀♫♀♫☼↓|♀↑☺↑♦☻☺♀☻☺♣|§4||♫♀♫☼♀♫|☼↑♫☼♀♫♀♫☼↓|♀↑☺↑♦☻☺♀☻☺♣                |§4||♫♀♫☼♀♫|☼↑♫☼♀♫♀♫☼↓|♀↑☺↑♦☻☺♀☻☺♣  #######

#C:\Users\jose.villa\Documents\proyectos\RecomendacionPersonalizadaIsa

## leer la info histórica
ruta='C:\\Users\\jose.villa\\Documents\\proyectos\\RecomendacionPersonalizadaIsa\\Hist12MCapDispJul22Jun15042024_Col_.csv'   
Historico_fecha='202207202306-3'  
Tienda='LA MARINA COLIMA'                                                                                                               
df=pd.read_csv(ruta)
## TOODO EN EXCEL
Excel_resultadosGpoTA = pd.DataFrame()
Excel_resultadosGpoTB = pd.DataFrame()
Excel_resultadosTA = pd.DataFrame()
Excel_resultadosTB = pd.DataFrame()
df=df.sort_values(by='Cuenta')
##df=df.sort_values(by='Cuenta', ascending=False)
cuentas_unicas = df['Cuenta'].drop_duplicates().rename_axis('Eliminar').reset_index()
#print(cuentas_unicas)


#df[df.Cuenta == 216]
#df.head(n=60)


ccuenta = cuentas_unicas['Cuenta'][885]
ccuenta



##################################################################################################################

df_parte=df[df.Cuenta == ccuenta]
    #Bucar compras de la cuenta
históricoCompras=list(df_parte['CodGpoArt'].drop_duplicates())
    #Si la cuenta no tiene compras, no se encuentra en la base
productos_comprados=len(históricoCompras)

us_genero=df_parte.iloc[0, 1]
us_edad=df_parte.iloc[0, 2]
us_cappago=df_parte.iloc[0, 6]
        #print("Productos comprados por el usuario:")    
        #Buscar cuentas que hayan comprado los mismos productos que la cuenta insertada, ordenada por cuenta
df_registros=pd.DataFrame()
for i in range(productos_comprados):
            #print(históricoCompras[i])
    df_cuenta_productos=df[df.CodGpoArt == históricoCompras[i]]
    df_registros= pd.concat([df_registros, df_cuenta_productos], ignore_index=True)   
df_registros=df_registros.sort_values(by=['Cuenta'])
        #excluir la cuenta insertada
df_registros=df_registros[df_registros.Cuenta != ccuenta ].sort_values(by=['Cuenta'])
        #Dejar solo las cuentas que hayan comprado alguno de los productos comprados por la cuenta insertada
df_registros_b=pd.DataFrame(data=df_registros[['Cuenta','Genero', 'Edad','CapacidadPago']], columns=['Cuenta','Genero', 'Edad','CapacidadPago']).reset_index(drop=True)

df_registros_b2=df_registros_b
        # #Dejar solo las cuentas con el mismo genero
df_registros_b2=df_registros_b2[df_registros_b2['Genero'] == us_genero]
        #Dejar solo las cuentas con rango de edad +-4 años
df_registros_b2=df_registros_b2[(df_registros_b2['Edad'] >= us_edad-4) & (df_registros_b2['Edad'] <= us_edad+4)]
        #Dejar solo las cuentas con rango de CAPPAGO +-200 
df_registros_b2=df_registros_b2[(df_registros_b2['CapacidadPago'] >= us_cappago-200) & (df_registros_b2['CapacidadPago'] <= us_cappago+200)]
        #seleccionar solo las cuentas para buscar todos los demás productos que haya comprado
cuentasABuscar=df_registros_b2.Cuenta.unique()

noCuentas = len(cuentasABuscar)

# Buscar en las cuentas

# Seleccionar productos

In [4]:
#seleccionar todos los productos que esas cuentas han comprado
mask = df['Cuenta'].isin(cuentasABuscar)
productosSeleccionador = df[mask]
        #excluir los productos que ya compró la cuenta insertada
mask = productosSeleccionador['CodGpoArt'].isin(históricoCompras)
productosSeleccionador2 = productosSeleccionador[~mask]
        #quitar los duplicados de cuenta-GpoArt para sacar los cálculos con una aparición por cuenta
productosSeleccionador2_sinduplicados=productosSeleccionador2.drop_duplicates()


        #ordernar por soporte de cada GpoArt
GpoArtDepa=productosSeleccionador2_sinduplicados['CodGpoArt-CodDepa'].value_counts(ascending=False)
GpoArtDepa

        #print("GpoArt-Depa: \n",GpoArtDepa.head(10))
        #ordenar por soporte cada Depa|
productosSeleccionador3=productosSeleccionador2_sinduplicados.copy() 
productosSeleccionador3['SoloDepa'] = productosSeleccionador2_sinduplicados['CodGpoArt-CodDepa'].str.split('-').str[1]

productosSeleccionador3
        #Aqui se puede hacer el calculo de soporte usando GpoArtDepa

        #separar Gpo y Dep en dos campos
productosSeleccionadorGpo=productosSeleccionador3[['Cuenta','CodGpoArt','SoloDepa']]
productosSeleccionadorGpo['SoloGpo'] = productosSeleccionadorGpo['CodGpoArt']
Depas_TA=[13,10,37,38,15,8,16,35,30,44,25]#['ALTA JOYERIA','COLCHONES','COMPUTO','LENTES SOLARES','LINEA BLANCA','MOTOCICLETAS','MUEBLERIA','RELOJERIA','TELEFONIA CELULAR','VIDEO','VIDEOJUEGOS']
productosSeleccionadorGpo['SoloDepa']=productosSeleccionadorGpo['SoloDepa'].astype(int)
maskGpo_TA=productosSeleccionadorGpo['SoloDepa'].isin(Depas_TA)
productosseleccionadosGpo_TA = productosSeleccionadorGpo[maskGpo_TA]
productosseleccionadosGpo_TB = productosSeleccionadorGpo[~maskGpo_TA]


Gpo_TA=pd.DataFrame(productosseleccionadosGpo_TA['CodGpoArt'].value_counts(ascending=False))
Gpo_TA = Gpo_TA.rename_axis('CodGpoArt').reset_index()
Gpo_TA['Support'] = Gpo_TA['count']/len(productosseleccionadosGpo_TA)



top_gpo = Gpo_TA.nlargest(2, 'count')

#print(top_gpo.columns)

cuentas_distintas = {}
for cod_gpo in top_gpo['CodGpoArt']:
    cuentas_distintas[cod_gpo] = productosseleccionadosGpo_TA[productosseleccionadosGpo_TA['CodGpoArt'] == cod_gpo]['Cuenta'].nunique()

top_gpo
productosseleccionadosGpo_TA
Gpo_TA

conteos = productosseleccionadosGpo_TA.groupby(['CodGpoArt', 'Cuenta']).size().reset_index(name='Cantidad')
productosseleccionadosGpo_TA = productosseleccionadosGpo_TA.merge(conteos, on=['CodGpoArt', 'Cuenta'], how='left')
#print(productosseleccionadosGpo_TA)

productos_filtrados = productosseleccionadosGpo_TA[productosseleccionadosGpo_TA['CodGpoArt'].isin(top_gpo['CodGpoArt'])]
productos_filtrados
conteos = productos_filtrados.groupby(['CodGpoArt', 'Cuenta']).size().reset_index(name='Cantidad')
productosseleccionadosGpo_TA = productosseleccionadosGpo_TA.merge(conteos, on=['CodGpoArt', 'Cuenta'], how='left')
#print(productosseleccionadosGpo_TA)




############################################################

cuentas_distintas


mayor_count_cod_gpo = top_gpo.iloc[0]['CodGpoArt']
total_cuentas_mayor = productosseleccionadosGpo_TA[productosseleccionadosGpo_TA['CodGpoArt'] == mayor_count_cod_gpo]['Cuenta'].nunique()

mayor_count_cod_gpo
total_cuentas_mayor

#Gpo_TA['Confidence'] =  / total_cuentas_mayor

#Gpo_TA['Confidence'] = gente con dos articulos de mayor freq / total de gente con articulo de mayor frecuencia

proporcion = cuentas_distintas[mayor_count_cod_gpo] / total_cuentas_mayor if total_cuentas_mayor > 0 else 0
proporcion





# Suponiendo que ya tienes el DataFrame Gpo_TA
max_count = Gpo_TA['count'].max()  # Encuentra el valor máximo de la columna 'count'
max_count
cod_gpo_art_max = Gpo_TA[Gpo_TA['count'] == max_count]['CodGpoArt']
#cod_gpo_art_max = Gpo_TA[Gpo_TA['count'] == max_count]['Gpo']


print(cod_gpo_art_max)



Gpo_TB=pd.DataFrame(productosseleccionadosGpo_TB['CodGpoArt'].value_counts(ascending=False))
Gpo_TB = Gpo_TB.rename_axis('CodGpoArt').reset_index()
#Gpo_TB = Gpo_TB.rename_axis('Gpo').reset_index()
Gpo_TB['Support'] = Gpo_TB['count']/len(productosseleccionadosGpo_TB)


        #quitar los duplicados de cuenta-Dep para sacar los cálculos con una aparición por cuenta
productosSeleccionador4=productosSeleccionador3[['Cuenta', 'SoloDepa']].drop_duplicates()
productosSeleccionador4['SoloDepa']=productosSeleccionador4['SoloDepa'].astype(int)
        #quitar los duplicados de cuenta-SoloDepa para sacar los cálculos con una aparición por cuenta
mask_TA=productosSeleccionador4['SoloDepa'].isin(Depas_TA)
productosseleccionados_TA = productosSeleccionador4[mask_TA]
productosseleccionados_TB = productosSeleccionador4[~mask_TA]


0    301239
Name: CodGpoArt, dtype: int64


# Calculo de la confianza para cada uno de los grupo de arículos.
Compute the confidence, lift and support of a product: let $G(p)$ the enviroment of the client $p$.

In [49]:


for c in list(range(0, 3000)):
    ccuenta = cuentas_unicas['Cuenta'][c]
    df_parte = df[df.Cuenta == ccuenta]
    históricoCompras = list(df_parte['CodGpoArt'])#.drop_duplicates())
    productos_comprados = len(históricoCompras)

    if productos_comprados > 0:
        us_genero = df_parte.iloc[0, 1]
        us_edad = df_parte.iloc[0, 2]
        us_cappago = df_parte.iloc[0, 6]

        df_registros = pd.DataFrame()
        for i in range(productos_comprados):
            df_cuenta_productos = df[df.CodGpoArt == históricoCompras[i]]
            df_registros = pd.concat([df_registros, df_cuenta_productos], ignore_index=True)

        df_registros = df_registros.sort_values(by=['Cuenta'])
        df_registros = df_registros[df_registros.Cuenta != ccuenta].sort_values(by=['Cuenta'])

        df_registros_b = pd.DataFrame(data=df_registros[['Cuenta', 'Genero', 'Edad', 'CapacidadPago']],
                                      columns=['Cuenta', 'Genero', 'Edad', 'CapacidadPago']).reset_index(drop=True)

        # Filter by gender, age range, and payment capacity range
        df_registros_b2 = df_registros_b[(df_registros_b['Genero'] == us_genero) &
                                         (df_registros_b['Edad'] >= us_edad - 4) &
                                         (df_registros_b['Edad'] <= us_edad + 4) &
                                         (df_registros_b['CapacidadPago'] >= us_cappago - 200) &
                                         (df_registros_b['CapacidadPago'] <= us_cappago + 200)]

        cuentasABuscar = df_registros_b2.Cuenta.unique()
        productosSeleccionador = df[df['Cuenta'].isin(cuentasABuscar)]
        productosSeleccionador2 = productosSeleccionador[productosSeleccionador['CodGpoArt'].isin(históricoCompras)]
        productosSeleccionador2_sinduplicados = productosSeleccionador2.drop_duplicates()

        productosSeleccionador3 = productosSeleccionador2_sinduplicados.copy()
        productosSeleccionador3['SoloDepa'] = productosSeleccionador3['CodGpoArt-CodDepa'].str.split('-').str[1].astype(int)

        Depas_TA = [13, 10, 37, 38, 15, 8, 16, 35, 30, 44, 25]
        productosseleccionadosGpo_TA = productosSeleccionador3[~productosSeleccionador3['SoloDepa'].isin(Depas_TA)]
        productosseleccionadosGpo_TB = productosSeleccionador3[~productosSeleccionador3['SoloDepa'].isin(Depas_TA)]

        Gpo_TA = pd.DataFrame(productosseleccionadosGpo_TA['CodGpoArt'].value_counts(ascending=False))
        Gpo_TA = Gpo_TA.rename_axis('CodGpoArt').reset_index()
        total_count = Gpo_TA['count'].sum()
        Gpo_TA['Support'] = Gpo_TA['count'] / total_count
        Gpo_TA['Tamano'] = Gpo_TA['count']

        total =[]
        for i in Gpo_TA['count']:
            total.append(total_count)
        Gpo_TA['TotEnt'] = total

        Gpo_TB = pd.DataFrame(productosseleccionadosGpo_TB['CodGpoArt'].value_counts(ascending=False))
        Gpo_TB = Gpo_TB.rename_axis('CodGpoArt').reset_index()
        total_countB = Gpo_TB['count'].sum()
        Gpo_TB['Support'] = Gpo_TB['count'] / total_countB
        # Get the top two most frequent articles in each group
        most_frequent_TA = productosseleccionadosGpo_TA['CodGpoArt'].value_counts().nlargest(2).index
        most_frequent_TB = productosseleccionadosGpo_TB['CodGpoArt'].value_counts().nlargest(2).index

        # Define a function to calculate conditional probability P(A|B) for each article
        def calculate_confidence(df_group, most_frequent_articles):
            confidence_values = []
            reference_articles = []
            account_both_articles =[]
            total_comun =[]
            common_both_article =[]
            total_common_values =[]
            
            unique_articles = df_group['CodGpoArt'].unique()
            for article in unique_articles:
                confidences = [] 
                reference_article = None
                
                
                for common_article in most_frequent_articles:
                    accounts_with_common = df_group[df_group['CodGpoArt'] == common_article]['Cuenta']
                    total_with_common = len(accounts_with_common)
                    accounts_with_both = df_group[(df_group['Cuenta'].isin(accounts_with_common)) & 
                                                 (df_group['CodGpoArt'] == article)]['Cuenta'].count()
                    confidence = accounts_with_both / total_with_common if total_with_common > 0 else 0
                    
                    confidences.append(confidence)
                    account_both_articles.append(accounts_with_both)
                    total_comun.append(total_with_common)
                    

                if confidences:
                    max_confidence = max(confidences)
                    max_confidence_index = confidences.index(max_confidence)
                    reference_article = most_frequent_articles[max_confidence_index]
                else:
                    max_confidence = 0
                    #reference_article = None
                
                confidence_values.append(max_confidence)
                reference_articles.append(reference_article)
#################################################################################
                if account_both_articles:
                    max_account = max(account_both_articles)  # Find the maximum value
                    max_bot_article_index = account_both_articles.index(max_account)  # Find the index in account_both_articles
                
                else:
                    max_account = 0
                
                common_both_article.append(max_account)

                if total_comun:
                    max_tot = max(total_comun)  # Find the maximum value
                    max_total_tot = total_comun.index(max_tot)  # Find the index in account_both_articles
                
                else:
                    max_tot = 0
                    
                
                total_common_values.append(max_tot)


                
                

            confidence_series = pd.Series(confidence_values, index=unique_articles).reindex(df_group['CodGpoArt']).fillna(0).reset_index(drop=True)
            reference_article_series = pd.Series(reference_articles, index=unique_articles).reindex(df_group['CodGpoArt']).fillna("None").reset_index(drop=True)
            account_both_articles_series = pd.Series(common_both_article, index=unique_articles).reindex(df_group['CodGpoArt']).fillna(0).reset_index(drop=True)
            account_totgpo_series = pd.Series(total_common_values, index=unique_articles).reindex(df_group['CodGpoArt']).fillna(0).reset_index(drop=True)

            #total_comun_series = pd.Series(total_comun, index=unique_articles).reindex(df_group['CodGpoArt']).fillna(0).reset_index(drop=True)
            #accounts_with_both_series = pd.Series(accounts_with_both_values, index=unique_articles).reindex(df_group['CodGpoArt']).fillna(0).reset_index(drop=True)
            #total_with_common_series = pd.Series(total_with_common_values, index=unique_articles).reindex(df_group['CodGpoArt']).fillna(0).reset_index(drop=True)

            return confidence_series, reference_article_series,account_both_articles_series,account_totgpo_series

        # Calculate confidence for both TA and TB groups
        Gpo_TA['Confidence'], Gpo_TA['ReferenceArticle'],Gpo_TA['articulos_comun'],Gpo_TA['totcomun']= calculate_confidence(productosseleccionadosGpo_TA, most_frequent_TA)
        Gpo_TB['Confidence'], Gpo_TB['ReferenceArticle'],Gpo_TB['articulos_comun'],Gpo_TB['totcomun']= calculate_confidence(productosseleccionadosGpo_TB, most_frequent_TB)

        Gpo_TA['Lift'] = Gpo_TA['Confidence']/Gpo_TA['Support']
        Gpo_TB['Lift'] = Gpo_TB['Confidence']/Gpo_TB['Support']
        # Display results
        print("Group TA - Top Articles:")
        print(Gpo_TA[['CodGpoArt', 'Tamano','Support', 'TotEnt','articulos_comun','totcomun','Confidence','Lift','ReferenceArticle']])

        print("\nGroup TB - Top Articles:")
        print(Gpo_TB[['CodGpoArt', 'Support', 'Confidence', 'Lift','ReferenceArticle']])



        Gpo_TA = Gpo_TA[['CodGpoArt', 'Support','Tamano','TotEnt','articulos_comun','totcomun','Confidence','Lift','ReferenceArticle']].head(10).rename_axis('Orden').reset_index()
        Gpo_TB = Gpo_TB[['CodGpoArt', 'Support','articulos_comun','Confidence','Lift','ReferenceArticle']].head(10).rename_axis('Orden').reset_index()

        # Concatenate the results for Gpo_TA, Gpo_TB, TA, and TB with the corresponding columns
        Excel_resultadosGpoTA = pd.concat([Excel_resultadosGpoTA, pd.DataFrame({
            'AnalisisID': Historico_fecha,
            'Cuenta': ccuenta,
            'Negocio': 1,
            'CodGpoArt': Gpo_TA['CodGpoArt'],
            'Orden': Gpo_TA['Orden'],
            'Tamano': Gpo_TA['Tamano'],
            'TotEnt': Gpo_TA['TotEnt'],
            'ArtComun' : Gpo_TA['articulos_comun'],
            'Totalcomun':Gpo_TA['totcomun'],
            'Support': Gpo_TA['Support'],
            'Confianza':Gpo_TA['Confidence'],
            'Lift':  Gpo_TA['Lift'],
            'ArtReeferencia':Gpo_TA['ReferenceArticle'],
            'EtiquetaDepa': 'Ticket Alto',
            'Region': Tienda
        })], ignore_index=True)

        Excel_resultadosGpoTB = pd.concat([Excel_resultadosGpoTB, pd.DataFrame({
            'AnalisisID': Historico_fecha,
            'Cuenta': ccuenta,
            'Negocio': 1,
            'CodGpoArt': Gpo_TB['CodGpoArt'],
            'Orden': Gpo_TB['Orden'],
            'Support': Gpo_TB['Support'],
            'Confianza':Gpo_TB['Confidence'], 
            'Lift':  Gpo_TB['Lift'],
            'ArtReeferencia':Gpo_TB['ReferenceArticle'],
            'EtiquetaDepa': 'Ticket Bajo',
            'Region': Tienda
        })], ignore_index=True)





Excel_resultadosGpoTA.to_csv(f'C:\\Users\\jose.villa\\Documents\\proyectos\\RecomendacionPersonalizadaIsa\\Excel_ResultadosGpoTA_{Tienda}.csv', index=False)
Excel_resultadosGpoTB.to_csv(f'C:\\Users\\jose.villa\\Documents\\proyectos\\RecomendacionPersonalizadaIsa\\Excel_ResultadosGpoTB_{Tienda}.csv', index=False)




Group TA - Top Articles:
   CodGpoArt  Tamano  Support  TotEnt  articulos_comun  totcomun  Confidence  \
0      60116       1      0.5       2                1         1         1.0   
1     120201       1      0.5       2                1         1         1.0   

   Lift  ReferenceArticle  
0   2.0             60116  
1   2.0            120201  

Group TB - Top Articles:
   CodGpoArt  Support  Confidence  Lift  ReferenceArticle
0      60116      0.5         1.0   2.0             60116
1     120201      0.5         1.0   2.0            120201
Group TA - Top Articles:
   CodGpoArt  Tamano  Support  TotEnt  articulos_comun  totcomun  Confidence  \
0      50201       4    0.500       8                2         4         1.0   
1      60503       2    0.250       8                4         4         1.0   
2     120203       1    0.125       8                4         4         1.0   
3      30105       1    0.125       8                4         4         1.0   

   Lift  ReferenceArticl

In [ ]:
if confidences:
                    max_confidence = max(confidences)
                    max_confidence_index = confidences.index(max_confidence)
                    reference_article = most_frequent_articles[max_confidence_index]
                else:
                    max_confidence = 0
                    #reference_article = None
                
                confidence_values.append(max_confidence)
                reference_articles.append(reference_article)
#################################################################################
                if account_both_articles:
                    max_account = max(account_both_articles)  # Find the maximum value
                    max_bot_article_index = account_both_articles.index(max_account)  # Find the index in account_both_articles
                
                else:
                    max_account = 0
                
                common_both_article.append(max_account)

                if total_comun:
                    max_tot = max(total_comun)  # Find the maximum value
                    max_total_tot = total_comun.index(max_tot)  # Find the index in account_both_articles
                
                else:
                    max_tot = 0
                    
                
                total_common_values.append(max_tot)